<i>Le programme contenu dans ce Notebook permet de jouer au jeu de l'Hexapion contre une intelligence artificielle (apprentissage par renforcement). 
</i>

<span style="color: #9317B4"> Pour exécuter une saisie Python, sélectionner la cellule et valider avec </span><span style="color: #B317B4"><strong>SHIFT+Entrée</strong></span>.


In [1]:
"""
Jeu de l'Hexapion 
Ce programme :
- permet à deux joueurs humains de s'affronter
- permet à un humain d'affronter une IA avec apprentissage par renforcement
Les variables globales N et M correspondent aux dimensions du plateau de jeu (largeur N=3 et hauteur M=3 par défaut)
"""

from time import sleep

N=3 #N :largeur du plateau de jeu
M=3 #M :longueur du plateau de jeu

# 0 1 ... N (<-ligne des noirs)
# 1
# .
# .
# M-1 ...   (<-ligne des blancs) 

from random import choice
from copy import deepcopy

class position:
    
    def __init__(self,grille=[ [False if j==0 else True if j==M-1 else None for i in range(N)] for j in range(M)],tour_de_jeu=True):
        """
        Création d'une position.
        Par défaut, position de départ avec tour de jeu aux blancs
        (conventions : True = blanc, False = noir)
        """
        
        self.grille = grille 
        
        self.tour_de_jeu = tour_de_jeu
        
    def __eq__(self,other): #symbole ==
        """
        indique si deux positions sont identiques
        """
        return self.tour_de_jeu == other.tour_de_jeu and self.grille == other.grille
        
    def __invert__(self): #symbole ~ devant la position    
        """
        renvoie la position symétrique (avec même tour de jeu)
        """
        sym = position()
        return position([ ligne[::-1] for ligne in self.grille ],self.tour_de_jeu)
        
    def coups(self):
        """
        renvoie la liste des positions suivantes possibles
        """
        liste_coups=[]
        
        #on parcourt toutes les cases de la grille
        for m in range(len(self.grille)):
            for n in range(len(self.grille[m])):
                
                #variable pour le sens de mouvement
                sens = -1 if self.tour_de_jeu else 1
                #on repère les pions de la couleur qui joue
                
                if self.grille[m][n] == self.tour_de_jeu:

                    #on teste si le pion peut prendre ou avancer (trois directions possibles)
                    for dir in [-1,0,1]:
                        
                        #test enlevé0 <= m+sens <M and
                        
                        if 0<= n+dir <N and self.grille[m+sens][n+dir] == (None if dir==0 else not self.tour_de_jeu):
                            coup=position(deepcopy(self.grille),not self.tour_de_jeu) #on duplique la position (avec changement de tour de jeu)
                            coup.grille[m][n] = None #on vide la case du pion déplacé
                            coup.grille[m+sens][n+dir] = self.tour_de_jeu #on place le pion dans sa nouvelle position
                            liste_coups.append(coup) #on ajoute le coup à la liste des coups possibles
                            

                            
        return liste_coups
        
    def gain(self):
        """
        renvoie le joueur gagnant:
        True  -> blanc
        False -> noir
        None  -> pas encore de vainqueur
        """
        
        #si un pion a atteint la dernière rangée, il y a un vainqueur
        if False in self.grille[M-1]: return False
        if True in self.grille[0]: return True
        
        #s'il n'y a plus de coups possibles, il y a un vainqueur
        #(NB: ce test gère aussi a fortiori le cas où il n'y a plus de jeton à jouer)
        if self.coups()==[]: return not self.tour_de_jeu
        
        #sinon il n'y a pas encore de vainqueur
        return None
            
    def genere_lignes_affichage(self):
        """
        génère une liste contenant chaque ligne d'affichage de la position dans la console
        "O" = pion blanc 
        " " = case vide
        "X" = pion noir
        ─┐┌└┘│
        
        Pour permettre une optimisation de l'affichage si on veut juxtaposer des grilles, 
        la méthode renvoie la liste des lignes (lignes sous forme de str)
        N :largeur du plateau de jeu
        M :longueur du plateau de jeu
        """
        Aff_grille = ["┌"+"─"*N+"┐"]
        for ligne in self.grille:
            Aff_ligne = "│"
            for case in ligne:
                Aff_ligne += " " if case is None else "O" if case else "X"            
            Aff_ligne += "│"
            Aff_grille.append(Aff_ligne)
                
            
        Aff_grille.append("└"+"─"*N+"┘")
        Aff_grille.append("Tour:"+" "*(N-3)) 
        Aff_grille.append(("Blanc" if self.tour_de_jeu else "Noir ")+" "*(N-3))
        return Aff_grille  
        
    def __str__(self):
        """
        Conversion en chaîne de caractère
        Permet d'utiliser la syntaxe print pour un objet position
        """    
        Aff=""
        for ligne in self.genere_lignes_affichage(): Aff += ligne+"\n"
        return Aff

class arbre:
    
    def __init__(self,config=position(),liste_coups_suivants=None):
        
        #position étudiée (objet de type position)
        self.configuration  = config      
        
        #coups suivants (liste d'objets de type arbre)
        #avec pour conventions:
        # None : n'a pas été étudié
        # [] : pas de coups suivants (la partie est finie)
                
        self.coups_suivants = liste_coups_suivants 
         
        
    def evaluer(self):
        """
        renvoie l'évaluation de la position de l'arbre en fonction des branches suivantes
        True :  blanc gagnant
        False : noir gagnant
        None :  issue inconnue
        """
        
        # si la position n'a jamais été atteinte,
        # on considère l'issue comme incertaine
        if self.coups_suivants==None : 
            return None
        
        # sinon on teste si c'est une position de fin de jeu,
        # et le cas échéant on renvoie l'issue
        gain_direct=self.configuration.gain()
        if gain_direct!=None:
            return gain_direct
        
        #on regarde qui a la main
        #pour ensuite tester les arbres fils
        t = self.configuration.tour_de_jeu
        
        #si le joueur qui la main a UNE SEULE position gagnante, alors il gagne
        
        for c in self.coups_suivants:
            if c.configuration.gain()==t:
                return t
        
        #si l'autre joueur (qui n'a pas la main) a TOUTES les positions gagnantes, alors il gagne
        All_gain_adv=True
        for c in self.coups_suivants:
            if c.configuration.gain()!=(not t):
                All_gain_adv=False
                break
        if All_gain_adv:
            return (not t)
        
        #dans tous les autres cas, l'issue est inconnue
        return None
        
    def largeur(self):
        """
        Renvoie la largeur de l'arbre, branches comprises.
        Méthode de classe utile pour l'affichage d'un arbre
        """    
        if not self.coups_suivants:
            return 1
        return sum([branches.largeur() for branches in self.coups_suivants])    
    
def partie(ordi=None,arb=arbre()):
    """
    Fonction qui permet de s'affronter au jeu de l'Hexapion
    ordi -> None : affrontement de deux joueurs humains
         -> True : l'ordinateur joue les blancs (coups au hasard)
         -> False: l'ordinateur joue les noirs  (coups au hasard)

    """
    
    print("┌────────────────────────────────────────────────┐ ")
    print("│Nouvelle partie                                 │ ")   
    print("│"+48*" "+"│ ")
    print("│Blanc :"+ ("ordi  " if ordi==True  else "humain")+" "*35+"│")
    print("│Noir  :"+ ("ordi  " if ordi==False else "humain")+" "*35+"│")
    print("└────────────────────────────────────────────────┘\n")
    
    curseur = arb
            
    # Affichage de la position courante
    print("\n_________________________________\nPosition actuelle:\n")
    print(curseur.configuration)
        
    
    # Tant que personne n'a gagné
    while curseur.configuration.gain() is None:
        
        # Si les coups suivants n'ont jamais été atteint 
        # on les ajoute dans l'arbre
        if curseur.coups_suivants==None:
            curseur.coups_suivants= [ arbre(config=c) for c in curseur.configuration.coups()]
        
        
        
        # Si c'est l'ordinateur qui doit jouer
        if curseur.configuration.tour_de_jeu==ordi:
            sleep(0.5)
            input("Valider pour que l'ordinateur joue")
            # L'ordinateur choisit un coup gagnant s'il en connait un
            a_joue=False
            for c in curseur.coups_suivants:
                if c.evaluer()==ordi:
                    curseur=c
                    a_joue=True
                    break
            # Si l'ordinateur n'a trouvé aucun coup gagnant,
            # il choisit au hasard parmi les coups à l'issue incertaine
            if not a_joue:
                try:
                    curseur = choice( [c for c in curseur.coups_suivants if c.evaluer() != (not ordi) ])
                except:
                    # gestion du cas où l'ordi n'a que des coups perdants,
                    # il choisit alors un coup au hasard
                    curseur = choice(curseur.coups_suivants)
            
        
        else:
            #Sinon (c'est l'humain qui joue)
        
            # Message requête
            print("Choisir un coup:")
            
            # Affichage des n° des coups possibles
            
            Affichage= [ curseur.coups_suivants[k].configuration.genere_lignes_affichage() for k in range(len(curseur.coups_suivants)) ]
                    
            ligne=""
            for j in range(len(curseur.coups_suivants)):
                ligne += str(j)+" "*(N+4)
            print(ligne)
            
            # Affichage des coups possibles
            
            for j in range(len(Affichage[0])):
                ligne=""
                for k in range(len(Affichage)): ligne += Affichage[k][j]+"   " 
                print(ligne)
            
            # Attente d'un choix de coup valide
            
            while True:
                try:
                    sleep(0.5)
                    choix = input("\nChoix:")
                    curseur = curseur.coups_suivants[int(choix)]
                    break
                except:
                    print("Saisie invalide.")
        
        # Affichage de la position courante
        print("\n_________________________________\nPosition actuelle:\n")
        print(curseur.configuration)
    
    #on indique dans l'arbre que la position du curseur
    #est une position finale du jeu
    curseur.coups_suivants=[]    
        
    # Renvoie l'indication du joueur gagnant et l'arbre actualisé   
    return curseur.configuration.gain() , arb
    

def serie_parties(ordi=None):
    """
    Fonction qui répète des parties au jeu de l'Hexapion
    ordi -> None : affrontement de deux joueurs humains
         -> True : l'ordinateur joue les blancs (coups au hasard)
         -> False: l'ordinateur joue les noirs  (coups au hasard)
    arbre -> arbre d'apprentissage de l'IA
    """
    
    print("┌────────────────────────────────────────────────┐ ")
    print("│            --- Jeu de l'Hexapion ---           │ ")
    print("│"+48*" "+"│ ")
    if ordi!=None:
        print("│ > Ordinateur avec IA par renforcement          │ ")   
    print("│ > Série de parties avec statistiques de gains  │ ")      
    print("│"+48*" "+"│ ")
    print("│Blanc :"+ ("ordi  " if ordi==True  else "humain")+" "*35+"│")
    print("│Noir  :"+ ("ordi  " if ordi==False else "humain")+" "*35+"│")
    print("└────────────────────────────────────────────────┘\n")
    
    arb = arbre()
    L=[] # liste des gains
    
    #boucle infinie de répétition de parties
    while True:
        gain,arb = partie(ordi,arb)
        L.append(gain)
        print("Partie terminée. Vainqueur : ","Blanc" if gain else "Noir")
        print("_____________________________________")
        print("Statistiques actuelles :")
        print("_____________________________________")
        print(" -> Historique des gains des parties antérieures :")
        chaine="    "
        for elt in L:
            chaine+= "B" if elt else "N"
        print(chaine)
        
        print(" -> Nombre de parties jouées : ",len(L))
        print(" -> Nombre de gains blancs :",L.count(True))
        print(" -> Nombre de gains noirs :",L.count(False))
        sleep(0.5)
        if input("Valider pour faire une nouvelle partie\n(saisir q ou Q pour quitter)") in ["q","Q"]: break
    
print("""
Exécuter la fonction serie_parties pour démarrer.
serie_parties(ordi=False) permet d'affronter l'ordinateur qui joue les noirs.
serie_parties(ordi=True) permet d'affronter l'ordinateur qui joue les blancs.
serie_parties() permet à deux joueurs humains de s'affronter.
""")



Exécuter la fonction serie_parties pour démarrer.
serie_parties(ordi=False) permet d'affronter l'ordinateur qui joue les noirs.
serie_parties(ordi=True) permet d'affronter l'ordinateur qui joue les blancs.
serie_parties() permet à deux joueurs humains de s'affronter.



In [ ]:
# Exécuter cette cellule permet d'affronter une IA qui joue les noirs
serie_parties(ordi=False)

┌────────────────────────────────────────────────┐ 
│            --- Jeu de l'Hexapion ---           │ 
│                                                │ 
│ > Ordinateur avec IA par renforcement          │ 
│ > Série de parties avec statistiques de gains  │ 
│                                                │ 
│Blanc :humain                                   │
│Noir  :ordi                                     │
└────────────────────────────────────────────────┘

┌────────────────────────────────────────────────┐ 
│Nouvelle partie                                 │ 
│                                                │ 
│Blanc :humain                                   │
│Noir  :ordi                                     │
└────────────────────────────────────────────────┘


_________________________________
Position actuelle:

┌───┐
│XXX│
│   │
│OOO│
└───┘
Tour:
Blanc

Choisir un coup:
0       1       2       
┌───┐   ┌───┐   ┌───┐   
│XXX│   │XXX│   │XXX│   
│O  │   │ O │   │  O│   
│ OO│   │

In [ ]:
# Exécuter cette cellule permet d'affronter une IA qui joue les blancs
serie_parties(ordi=True)